In [1]:
import PIL
from PIL import Image
import imagehash

import pycamhd.pycamhd as camhd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd       

def diff_letters(a,b):
    if (len(a) != len(b)):
        return -1
    else:
        return sum ( a[i] != b[i] for i in range(len(a)) )

dbcamhd = (pd.read_json('dbcamhd.json', orient="records", lines=True)).set_index("filename", drop = False)
dbcamhd.tail()  

fileIDs = []
avgHash = []
dHash = []
pHash = []
wHash = []

In [2]:
from dask import delayed, compute

@delayed
def delayed_get_frame(filename, frame_number):
    frame = camhd.get_frame(filename, frame_number)
    return Image.fromarray(frame)

@delayed
def delayed_average_hash(image):
    return imagehash.average_hash(image)

@delayed
def delayed_phash(image):
    return imagehash.phash(image)

@delayed
def delayed_dhash(image):
    return imagehash.dhash(image)

@delayed
def delayed_whash(image):
    return imagehash.whash(image)

In [28]:
for filename in dbcamhd.loc[dbcamhd['frame_count']>2000]['filename'][0:1]:
    frameTotal = dbcamhd.frame_count[filename]
    #print(frameTotal)
    timestamp = dbcamhd.timestamp[filename]
    frames =  range(0,frameTotal,10)
    #print(frames)
    
    for i in frames:
        ID = timestamp +  i
        im = delayed_get_frame(filename, i)
        #avg = delayed_average_hash(im)
        #p = delayed_phash(im)
        #d = delayed_dhash(im)
        w = delayed_whash(im)
        
        #avgHash.append(avg)
        #pHash.append(p)
        #dHash.append(d)
        wHash.append(str(w))

In [5]:
from dask_kubernetes import KubeCluster
from dask.distributed import Client

cluster = KubeCluster(n_workers=1)
cluster.scale(30)
cluster

In [29]:
client = Client(cluster)
client

Client Scheduler: tcp://10.244.0.16:45123 Dashboard: /user/al3623/proxy/45949/status,Cluster Workers: 30 Cores: 60 Memory: 120.00 GB


In [30]:
wHashList = compute(*wHash)

In [33]:
print(len(wHashList))
print(len(frames))

2268
1133


In [ ]:
plt.rc('font', size=12)
fig, ax = plt.subplots();
fig.set_size_inches(14, 6);
fig.frameon = False
hb = ax.hexbin(frames, wHashList[1135:], vmin=0, vmax=1, bins='log', linewidths=0.25,
  gridsize=(225,4500), mincnt=1, cmap=plt.cm.BuPu)
fig.colorbar(hb)
ax.set_ylim([0, 8000])
ax.set_xlim([frames[0],frames[-1]])
ax.yaxis.grid(True)
ax.xaxis.grid(True)